# ДП. Два параметра

## Поиск количества путей

Допустим, у нас есть квадратное поле $a$ размером $n\times n$, состоящее из клеток. Какие-то клетки недостижимы и по ним ходить нельзя. 

Разрешённые ходы: вправо и вниз. Нужно найти, сколькими способами можно попасть из клетки $(0, 0)$ в клетку $(n−1, n−1)$

<img src='img/dp10.jpg'>

Для каждой клетки мы хотим знать - сколько путей ведёт к ней из старта, учитывая ограничения.

Пусть `dp[i][j]` - количество способов попасть в клетку $(i, j)$. Значит, если клетка достижима, то в нее можно придти либо сверху - из клетки $(i-1, j)$, либо слева - из $(i, j-1)$, то есть
$$dp[i][j]=dp[i-1][j]+dp[i][j-1]$$
Если клетка нелостижима, то ставим `dp[i][j]=0`

Особым случаем будут являться первая строка и первый столбец, так как в них в клетки можно придти только слева и справа соответственно

In [3]:
n = 5
grid = [
    '...#.',
    '.#.##',
    '....#',
    '#.#.#',
    '.....'
]

dp = [[0] * n for _ in range(n)]
dp[0][0] = 1

for j in range(1, n):
    if grid[0][j] == '.':
        dp[0][j] = dp[0][j - 1]

for i in range(1, n):
    if grid[i][0] == '.':
        dp[i][0] = dp[i - 1][0]

for i in range(1, n):
    for j in range(1, n):
        if grid[i][j] == '.':
            dp[i][j] = dp[i - 1][j] + dp[i][j - 1]

print(dp[n - 1][n - 1])


3


На рисунке отмечены все пути, по которым можно добраться до клетки $(n-1, n-1)$
<img src='img/dp11.jpg'>

## Ищем выгоду

Предыдущую задачу можно модифицировать. Пусть по клеткам разбросаны монеты разного достоинства (возможно, отрицательного). Задача: собрать максимальную сумму

<img src='img/dp12.jpg'>

Решение будет похожим на предыдущее. Нам нужно будет создать массив `dp` и насчитывать пути

Очевидно, что в клетку `dp[i][j]` мы можем придти либо из `dp[i-1][j]` (то есть сверху), либо из `dp[i][j-1]` (слева). Так как наша задача набрать максимальную сумму, значит
\begin{equation*}
\begin{aligned}
dp[i][j]&=\max(dp[i-1][j]+a[i][j];dp[i][j-1]+a[i][j])\\
&=\max(dp[i-1][j];\ dp[i][j-1])+a[i][j]
\end{aligned}
\end{equation*}

<img src='img/dp13.jpg'>

Получаем, что максимальная сумма, которую мы сможем получить - 16 денежек

In [5]:
n, m = 6, 5
a = [
    [0, -1, 3, -1, 4],
    [2, -2, 4, -2, 2],
    [2, -1, -2, -3, -4],
    [4, 3, 2, 1, 0],
    [2, 1, 2, -2, 0],
    [-1, 3, -1, 2, 0]
]

dp = [[0] * m for _ in range(n)]
dp[0][0] = a[0][0]

for j in range(1, m):
    dp[0][j] = dp[0][j - 1] + a[0][j]

for i in range(1, n):
    dp[i][0] = dp[i - 1][0] + a[i][0]

for i in range(1, n):
    for j in range(1, m):
        dp[i][j] = max(dp[i - 1][j], dp[i][j - 1]) + a[i][j]

print(dp[n - 1][m - 1])


16


# Наибольшая возрастающая подпоследовательность за $O(n\log n)$

Пусть имеется массив `a=[3, 2, 4, 3, 7, 8, 5, 6, 9]`. Снова создаем массив `dp`, но теперь в `dp[i]` будет лежать минимальное значение, на которое заканчивается последовтальность длины $i$
\begin{equation*}
dp=[-\infty, +\infty, +\infty, +\infty, \ldots, +\infty]
\end{equation*}


Начинаем проход по массиву `a`:
- Увидели `a[0]=3`. Значит, мы можем получить последовательность длины 1, которая заканчивается на 3, следовательно `dp[1]=3`
- Дальше смотрим на `a[1]=2`. То есть мы модем получить две последовтальности длины 1, но нам нужно взять $min$ элемент. Заменяем `dp[1]=2`
- Видим `a[2]=4`. Ставим `dp[2]=4`, так как мы можем образовать последовательность `2-4` длины 2
- Берем `a[3]=3`. Мы поменяем `dp[2]=3`, так как мы можем образовать последовательность `2-3` длины 2 и `3<4`, берем $\min$ элемент
- Далее к последовательности `2-3` мы модем дописать сначала `7`, а потом `8`
- Теперь мы наткнулись на `5`. Можно ли куда-то поставить этот элемент? Да, в последовательности `2-3-7-8` можно заменить семерку числом `5`
- Аналогично, заменяем восьмерку числом `6`
- И приписываем в конец последовательности число `9`

Получили последовательность `2-3-5-6-9`

Но как нам искать то место, где можно заменить уже найденное число текущим? Бинпоиск спешит на помощь. Поставим левый и правый указатели на начало и конец `dp` соответственно

In [10]:
a = [3, 2, 4, 3, 7, 8, 5, 6, 9]
def lis(a):
    dp = [2**20] * len(a)
    dp[0] = -2**20

    for key in a:
        l, r = 0, len(dp)
        while l + 1 < r:
            mid = (l + r) // 2
            if dp[mid] < key:
                l = mid
            else:
                r = mid
        if dp[r] > key:
            dp[r] = key
    
    return dp

print(lis(a))

[-1048576, 2, 3, 5, 6, 9, 1048576, 1048576, 1048576]


**Важно:** этот алгоритм не находит саму НВП, а только её длину. Для восстановления НВП нужно дополнительно хранить массив предков

In [12]:
print(lis([5, 4, 3, 2, 1])) # НВП состоит из одного элемента, но не обязательно из числа 1

[-1048576, 1, 1048576, 1048576, 1048576]


# Наибольшая общая подпоследовательность

Есть два слова, например, `abac` и `baac`. Задача: выбрать в этих словах буквы, не обязательно идущие подряд, так, чтобы убрав все остальные буквы получилось два одинаковых слова 

Создаем массив `dp`

<img src='img/dp14.jpg'>

Алгоритм:
- если символы одинаковы, то есть `s1[i]=s2[j]`, то `dp[i][j]=dp[i-1][j-1]+1`
- если символы разные, то `dp[i][j]=max(dp[i-1][j]; dp[i][j-1])`

<img src='img/dp15.jpg'>

In [13]:
s1, s2 = 'abac', 'baac'

n = len(s1)
m = len(s2)

dp = [[0] * (m + 1) for _ in range(n + 1)]

for i in range(1, n + 1):
    for j in range(1, m + 1):
        if s1[i - 1] == s2[j - 1]:
            dp[i][j] = dp[i - 1][j - 1] + 1
        else:
            dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

print(dp[n][m])

3
